In [1]:
import pandas as pd
import numpy as np

# ID3 与C4.5算法

### 零、决策树

一系列` if-then`规则, 对于决策树而言，每个非叶子节点就是一种属性，该节点的出边为该属性的各种可能的取值。而决策树的叶子节点是类别。

对于输入的一组数据集，生成一颗决策树可以使用ID3或C4.5算法，这两种算法的最大区别在于选取分类属性的指标不同，ID3是直接选取最大信息增益的属性作为根节点，而C4.5是选取信息增益比最大的属性。
同时，C4.5还有一些辅助步骤帮助生成更好的决策树。

### 一、信息熵

衡量信息的复杂程度和信息量的一个度量值，熵越大说明数据越复杂，所携带的信息越多，对与一组数据而言，熵的计算公式如下：

$$ S= \sum_{i=0}^{n} -p_i log_{2}{p_i } $$

数据有n个取值，而$p_i$ 是各个取值的概率。可以注意到，该公式是对于无序的数据而已的，也就是数据的交换不会改变这个熵值。比如`1,1,1,1,0,0,0,0`和`1,0,1,0,1,0,1,0`的熵是一样的。

下面是计算熵的函数，输入是一个数据序列。

In [44]:
from math import log
def Ent(series :pd.Series) ->float:
    a  =series.shape[0]
    ent=0.0
    for i in series.value_counts():
        ent-= i/a *log(i/a,2)
    return ent

上面的算法只针对一种属性取值类别所含的信息熵，假如该属性含多个取值，我们定义其信息量为：

$$ E= \sum_{i=0}^{n} c_i S_i $$
$c_i$ 是该取值占整体的比例，$S_i$是在该取值下的**类型**分布的信息熵。

计算信息熵的函数：


In [42]:
def getE(df : pd.DataFrame,index : str):
    classname=df.columns[-1]
    theP=0.0
    allcount=df.shape[0]
    for v in df[index].unique():
        subdf=df[df[index]==v]
        subcount=subdf.shape[0]
        theP+=(subcount / allcount)*Ent(subdf[subdf.columns[-1]])
    return theP

现在假如整个数据量类别的信息熵为$S$，那么选择某个属性后获得的信息熵为$E$，则该属性的信息增益为$S-E$，也就是说，选择该属性作为判断条件，可以为我们带来多少信息量。显然信息量越多越好，所以我们要选信息增益最大的属性。这就是ID3算法的选择条件。

考虑到一个数据集的$S$是不变的，所以要求最大的信息增益，其实是求拥有最小$E$的属性。

但是，每一个属性的取值分布同时也携带一定的信息量。

$$ S_i= \sum_{i=0}^{n} -p_i log_{2}{p_i } $$

$p_i $ 是该属性各个**取值**的概率。

也就是对于取值类别较多的属性而言，它携带的信息量也比较多。选择该属性所带来的信息增益一部分属于属性本身的，我们要进行扣除。也就得到该属性的信息增益比例。

$$ G=\frac{S-E}{S_i}$$

C4.5算法选择拥有最大信息增益比例$G$的属性作为根节点。同理对于同一个数据集而言，$S$也是不变的，所以我们只要求$\frac {E} {S_i}$ 为最小的就行 

In [24]:
def getS_i(df : pd.DataFrame,index : str):
    classname=df.columns[-1]
    theP=0.0
    allcount=df.shape[0]
    for v in df[index].unique():
        subdf=df[df[index]==v]
        subcount=subdf.shape[0]
        theP+=(subcount / allcount)*log(subcount / allcount,2)
    return theP

In [25]:
def getG(df : pd.DataFrame,index : str) ->float:
    return getE(df,index)/getS_i(df,index)

### 二、算法步骤

对于一个数据集,生成节点的算法如下：
* 1.如果没有属性就采用多数表决
* 2.如果属于同一个类，则算法停止，使用该标记做分类 
* 3.根据最大信息增益或最大信息增益比例的属性划分子数据集，对每个数据集生成子节点

代码实现如下（先就ID3 做例子）：

In [31]:
def gID3Tree(df):
    
    if(ispure(df)): #无属性
        return gPureNode(df)
    
    if(issame(df)): #同一类
        return gSameNode(df)
        
    maxindex=getMaxIndex(df,getE)
    # 划分
    subdfs=df.groupby(maxindex)
    
    #生成节点和子节点
    node={}
    node["分组属性"]=maxindex
    node["子节点"]=[]
    for subdf in subdfs:
        sdf=subdf[1]
        sdf.pop(maxindex)
        node["子节点"].append((subdf[0],gTree(sdf)))
        
    return node

可以看到代码的逻辑基本和算法描述一样，接下来一个一个实现功能函数就行

判断是不是空属性和多数表决比较简单：

In [32]:
def ispure(df):
    return df.shape[1]==1
print(ispure(pd.DataFrame(np.zeros((3,10)))))
print(ispure(pd.DataFrame(np.zeros((3,1)))))

False
True


In [33]:
def maxType(df):
    """
    多数表决
    实现得不够优雅
    """
    maxcount=-1
    maxtype=None
    for sdf in df.groupby(df.columns[0]):
        if(sdf[1].count()[0] >maxcount):
            maxcount=sdf[1].count()[0]
            maxtype=sdf[0]
    return maxtype
(maxType(pd.DataFrame(np.array([1,1,1,1,1,2,2,2,3]).reshape((9,1))))
,maxType(pd.DataFrame(np.array([1,1,1,1,2,2,2,2,3]).reshape((9,1))))
,maxType(pd.DataFrame(np.array([1,1,1,2,2,2,2,2,3]).reshape((9,1)))))

(1, 1, 2)

判断是不是同一个类别的看最后一列就行

In [34]:
def issame(df):
    return df[df.columns[-1]].unique().size==1
(issame(pd.DataFrame(np.array([4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]).reshape(9,2))),
 issame(pd.DataFrame(np.array([4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4]).reshape(9,2))))

(True, False)

对于这两种情况就是叶子节点了

In [35]:
def gPureNode(df):
    types=maxType(df)
    node={}
    node["type"]=types
    return node
def gSameNode(df):
    types=df[df.columns[-1]].values[0]
    node={}
    node["type"]=types
    return node;

取最大增益的列名

In [36]:
def getMaxIndex(df,getD):
    if (df.columns.size<=2):
        return df.columns[0]
    
    allindex=df.columns[:-1]
    maxindex=allindex[0]
    maxp=getD(df,maxindex)
    for i in allindex[1:0]:
        p=getD(df,maxindex)
        if(p<maxp):
            maxp=p
            maxindex=i
    return maxindex

下面是教材内的测试数据，为了输入方便，用数值代替了

In [37]:
outlook=[1,1,2,3,3,3,2,1,1,3,1,2,2,3]
temp=[1,1,1,2,3,3,3,2,3,2,2,2,1,2]
hum=[1,1,1,1,2,2,2,1,2,2,2,1,2,1]
windy=[0,1,0,0,0,1,1,0,0,0,1,1,0,0]
play=[0,0,1,1,1,0,1,0,1,1,1,1,1,0]
DataSet = list(zip(outlook,temp,hum,windy,play))
playdf = pd.DataFrame(data = DataSet, columns=["outlook","temp","hum","windy","play"])

In [38]:
playdf

,outlook,temp,hum,windy,play
0,1,1,1,0,0
1,1,1,1,1,0
2,2,1,1,0,1
3,3,2,1,0,1
4,3,3,2,0,1
5,3,3,2,1,0
6,2,3,2,1,1
7,1,2,1,0,0
8,1,3,2,0,1
9,3,2,2,0,1


测试一下信息熵函数有没有错,结果应该是0.694

In [45]:
getE(playdf,"outlook")

0.6935361388961918

In [46]:
tree=gID3Tree(playdf)
tree

{'分组属性': 'outlook',
 '子节点': [(1,
   {'分组属性': 'temp',
    '子节点': [(1, {'type': 0}),
     (2, {'分组属性': 'hum', '子节点': [(1, {'type': 0}), (2, {'type': 1})]}),
     (3, {'type': 1})]}),
  (2, {'type': 1}),
  (3,
   {'分组属性': 'temp',
    '子节点': [(2,
      {'分组属性': 'hum',
       '子节点': [(1, {'分组属性': 'windy', '子节点': [(0, {'type': 0})]}),
        (2, {'type': 1})]}),
     (3,
      {'分组属性': 'hum',
       '子节点': [(2,
         {'分组属性': 'windy',
          '子节点': [(0, {'type': 1}), (1, {'type': 0})]})]})]})]}

预测

测试数据集

In [139]:
DataSet = list([(3,3,2,0),(1,2,2,0)])
test = pd.DataFrame(data = DataSet, columns=["outlook","temp","hum","windy"])

In [140]:
test

,outlook,temp,hum,windy
0,3,3,2,0
1,1,2,2,0


预测函数

In [141]:
def predict(tree,data):
    for i in range(data.shape[0]):
        line=data.iloc[i]
        t=tree
        while True:
            if t.get("type") !=None:
                print(t["type"])
                break
            index=t["分组属性"]
            value=line[index]
            f=False
            
            for i in t["子节点"]:
                if(i[0])==value:
                    t=i[1]
                    f=True
                    break
            if f ==False:
                print("找不到属性",index,"为",value,"的分支")
                break

结果

In [142]:
predict(tree,test)

1
1


C4.5只需要使用G作为衡量指标即可

In [47]:
def gC45Tree(df):
    
    if(ispure(df)): #无属性
        return gPureNode(df)
    
    if(issame(df)): #同一类
        return gSameNode(df)
        
    maxindex=getMaxIndex(df,getG)
    # 划分
    subdfs=df.groupby(maxindex)
    
    #生成节点和子节点
    node={}
    node["分组属性"]=maxindex
    node["子节点"]=[]
    for subdf in subdfs:
        sdf=subdf[1]
        sdf.pop(maxindex)
        node["子节点"].append((subdf[0],gTree(sdf)))
        
    return node

In [48]:
gC45Tree(playdf)

{'分组属性': 'outlook',
 '子节点': [(1,
   {'分组属性': 'temp',
    '子节点': [(1, {'type': 0}),
     (2, {'分组属性': 'hum', '子节点': [(1, {'type': 0}), (2, {'type': 1})]}),
     (3, {'type': 1})]}),
  (2, {'type': 1}),
  (3,
   {'分组属性': 'temp',
    '子节点': [(2,
      {'分组属性': 'hum',
       '子节点': [(1, {'分组属性': 'windy', '子节点': [(0, {'type': 0})]}),
        (2, {'type': 1})]}),
     (3,
      {'分组属性': 'hum',
       '子节点': [(2,
         {'分组属性': 'windy',
          '子节点': [(0, {'type': 1}), (1, {'type': 0})]})]})]})]}